<a href="https://colab.research.google.com/github/lochieh19798/Data-analysis/blob/main/Figure2mod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Edition control

In [1]:
!git config --global user.name "lochieh19798"
!git config --global user.email "lochieh19798@gmail.com"

In [2]:
!git clone https://github.com/lochieh19798/data-analysis

Cloning into 'data-analysis'...
remote: Enumerating objects: 74, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (71/71), done.
Receiving objects: 100% (74/74), 4.98 MiB | 10.21 MiB/s, done.
remote: Total 74 (delta 36), reused 0 (delta 0), pack-reused 0 (from 0)
Resolving deltas: 100% (36/36), done.


In [3]:
# ╔═╡ Cell 0 ═════════════════════════════════════════════════════════════╗
# If you’re on Colab, install any non-preinstalled libraries:
!pip install -q catboost shap scikit-learn==1.4.2 matplotlib pandas numpy
!pip install -q --upgrade xgboost
!pip install -q shap --no-deps

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 122.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 MB 3.4 MB/s eta 0:00:00


In [4]:
from xgboost.sklearn import XGBModel
XGBModel.feature_weights = None
print("Patched XGBModel.feature_weights →", XGBModel.feature_weights)

Patched XGBModel.feature_weights → None


In [5]:
# ╔═╡ Cell 1 ═════════════════════════════════════════════════════════════╗
# Imports & global configuration
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
from IPython.display import display

from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import (
    OneHotEncoder,
    StandardScaler,
    OrdinalEncoder
)
from sklearn.impute import (
    KNNImputer,
    SimpleImputer
    # ← no IterativeImputer here
)
from sklearn.metrics import roc_auc_score, RocCurveDisplay

# Classifiers
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (
    ExtraTreesClassifier,
    AdaBoostClassifier,
    HistGradientBoostingClassifier
)
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

RANDOM_STATE = 42
plt.style.use("default")


In [6]:
# ╔═╡ Cell 2 – load data & build feature lists with missing‐% report ═════╝
from pathlib import Path
import pandas as pd

# 1. Data‐loading helper (as before)
DATA_PATH = "/content/data-analysis/CRYOANALYSIS.csv"
def load_and_prepare_data(csv_path: str = DATA_PATH) -> pd.DataFrame:
    df = pd.read_csv(csv_path)
    # (1) Recurrence flags
    df["Survival_time"] = pd.to_numeric(df["Survival_time"], errors="coerce")
    df = df.dropna(subset=["Survival_time", "Recurrence"])
    df["Recurrence_1yr"] = ((df["Survival_time"] <= 365) & (df["Recurrence"] == 1)).astype(int)
    df["Recurrence_2yr"] = ((df["Survival_time"] <= 730) & (df["Recurrence"] == 1)).astype(int)
    # (2) AF type one‐hot
    af_col = "Baseline AF Type(1=paroxysmal, 2=persistent)"
    df["AF_Parox"]   = (df[af_col] == 1).astype(int)
    df["AF_Persist"] = (df[af_col] == 2).astype(int)
    return df

# 2. Load the dataset
DATA_PATH = Path("/content/data-analysis/CRYOANALYSIS.csv")
assert DATA_PATH.exists(), f"{DATA_PATH} not found – upload it first!"
df = load_and_prepare_data(DATA_PATH)

# 3. Feature lists
cont = [
    "age", "BMI",
    "Baseline LVEF", "Baseline LAD", "CHA2DS2VASc score", "CHAD2 score", "AF_time_procedure",
    "Total no of ablation application number", "Mobility question (Baseline)",
    "Self-care question (Baseline)", "Usual activities question (Baseline)",
    "Pain/Discomfort question (Baseline)", "Anxiety/Depression question (Baseline)",
    "Visual analogue score: Your own health state today (Baseline)",
    "Total procedure time: Venous access to last cryoatheter removal (mins)",
    "Total fluoro time (mins)", "Energy duration LSPV", "Coldest Temperature LSPV",
    "Energy duration LIPV", "Coldest Temperature LIPV",
    "Energy duration RSPV", "Coldest temperature RSPV",
    "Energy duration RIPV", "Coldest Temperature RIPV",
    "Left atrial dwell time: time from first cryocatheter insertion to last cryocatheter removal (mins)",
    "Change in EQ 5D"
    # "Time to isolation LSPV", "Time to isolation LIPV", "Time to isolation RSPV","Time to isolation RIPV",
]

cat = [
    "Sex (F=1, M=0)", "Hypertension", "Diabetes", "HF", "CAD", "stroke",
    "History of TIA", "Subject taking Class I or III AAD at baseline (1=Yes, 0=No)",
    "LSPV isolated", "LIPV PV isolated", "RSPV Isolated", "RIPV isolated",
    "Were all targeted PVs isolated (Investigator)?",
    "AF_Parox", "AF_Persist",
    "CTI ablation",
    "Non-PVI ablation performed",
    "Was subject taking Class I or Class III AAD at procedure discharge?",
    "Mapping/navigational tools: Intracardiac echocardiography (ICE)",
    "Pre Procedural CT",
    "Ensite 3D mapping"
]

TARGET = "Recurrence_2yr"

# 4. Report missing‐value percentages for every feature
all_feats = cont + cat
missing_pct = df[all_feats].isna().mean() * 100
missing_report = (
    missing_pct
      .sort_values(ascending=False)
      .rename("missing_pct")
      .reset_index()
      .rename(columns={"index": "variable"})
)
print("📊 Missing Value Percentage per Feature:")
print(missing_report.to_string(index=False, float_format="%.1f%%"))

# 5. Gentle column check: warn & drop if still missing
missing_cont = [c for c in cont if c not in df.columns]
missing_cat  = [c for c in cat  if c not in df.columns]
if missing_cont or missing_cat:
    print("\n⚠️  WARNING – some expected columns are missing and will be skipped.")
    print("   Missing continuous :", missing_cont)
    print("   Missing categorical:", missing_cat)

cont = [c for c in cont if c in df.columns]
cat  = [c for c in cat  if c in df.columns]

# From here on, the rest of the notebook runs as before...


📊 Missing Value Percentage per Feature:
                                                                                          variable  missing_pct
                                                                                      Baseline LAD        32.7%
                                                                                   Change in EQ 5D        27.0%
                                                            Anxiety/Depression question (Baseline)        25.6%
                                                               Pain/Discomfort question (Baseline)        25.6%
                                                              Usual activities question (Baseline)        25.6%
                                                                     Self-care question (Baseline)        25.6%
                                     Visual analogue score: Your own health state today (Baseline)        25.6%
                                                                

In [7]:
# ╔═╡ Cell 3 – stratified 3-way split: train/test/validation ═══════════╗
from sklearn.model_selection import train_test_split

# 1) Sort by date if you still want a temporal ordering internally (optional)
df["ProcedureDate"] = pd.to_datetime(df["ProcedureDate"])
df_sorted = df.sort_values("ProcedureDate")

# 2) Extract features & target
X_all = df_sorted[cont + cat]
y_all = df_sorted[TARGET].astype(int)

# 3) Initial 20% hold-out for future validation (stratified by target)
X_temp, X_val, y_temp, y_val = train_test_split(
    X_all, y_all,
    test_size=0.20,
    stratify=y_all,
    random_state=RANDOM_STATE
)

# 4) Of the remaining 80%, take a 20% slice for an internal test set
X_train, X_test, y_train, y_test = train_test_split(
    X_temp, y_temp,
    test_size=0.20,
    stratify=y_temp,
    random_state=RANDOM_STATE
)

# 5) Report sizes and event rates
print("Split sizes:")
print(f"  • Train:      {len(y_train)}")
print(f"  • Test:       {len(y_test)}")
print(f"  • Validation: {len(y_val)}\n")

print("Event rates:")
print(f"  • Overall:    {y_all.mean():.2%}")
print(f"  • Train:      {y_train.mean():.2%}")
print(f"  • Test:       {y_test.mean():.2%}")
print(f"  • Validation: {y_val.mean():.2%}")


Split sizes:
  • Train:      179
  • Test:       45
  • Validation: 57

Event rates:
  • Overall:    19.22%
  • Train:      18.99%
  • Test:       20.00%
  • Validation: 19.30%


In [8]:
# ╔═╡ Cell 4 – Pre-processing pipelines ═══════════════════════════════════╗
# (Make sure to enable the experimental IterativeImputer)
from sklearn.experimental import enable_iterative_imputer
from sklearn.pipeline           import Pipeline
from sklearn.impute             import IterativeImputer, SimpleImputer
from sklearn.preprocessing      import StandardScaler, PowerTransformer, OneHotEncoder
from sklearn.compose            import ColumnTransformer

# Numeric pipeline: iterative imputation → power transform → scaling
numeric_pipe = Pipeline([
    ("impute", IterativeImputer(random_state=RANDOM_STATE)),
    ("power",  PowerTransformer(method="yeo-johnson")),
    ("scale",  StandardScaler())
])

# Categorical pipeline: fill missing with "Missing" → one-hot encode
categorical_pipe = Pipeline([
    ("impute", SimpleImputer(strategy="constant", fill_value="Missing")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

# Full preprocessor: apply to continuous and categorical lists
preprocessor = ColumnTransformer([
    ("num", numeric_pipe,       cont),
    ("cat", categorical_pipe,   cat)
], remainder="drop")


In [9]:
# ╔═╡ Cell 4b – preview IterativeImputer + PowerTransformer on numeric features ═══════╗
import warnings
from sklearn.exceptions import ConvergenceWarning

# (Optionally suppress FutureWarnings or ConvergenceWarnings)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)

# 1. Show missing counts before
num_train = X_train[cont].copy()
print("🔴 Missing before imputation (numeric features):")
print(num_train.isna().sum())

# 2. Fit & transform with your updated numeric_pipe
num_imputed = pd.DataFrame(
    numeric_pipe.fit_transform(num_train),
    columns=cont,
    index=num_train.index
)

# 3. Show missing counts after
print("\n🟢 Missing after IterativeImputer + PowerTransformer:")
print(num_imputed.isna().sum())

# 4. Display first few rows of the transformed DataFrame
display(num_imputed.head())


🔴 Missing before imputation (numeric features):
age                                                                                                    0
BMI                                                                                                    1
Baseline LVEF                                                                                         44
Baseline LAD                                                                                          57
CHA2DS2VASc score                                                                                      0
CHAD2 score                                                                                            0
AF_time_procedure                                                                                     18
Total no of ablation application number                                                                0
Mobility question (Baseline)                                                                          41
Self-ca

,age,BMI,Baseline LVEF,Baseline LAD,CHA2DS2VASc score,CHAD2 score,AF_time_procedure,Total no of ablation application number,Mobility question (Baseline),Self-care question (Baseline),...,Energy duration LSPV,Coldest Temperature LSPV,Energy duration LIPV,Coldest Temperature LIPV,Energy duration RSPV,Coldest temperature RSPV,Energy duration RIPV,Coldest Temperature RIPV,Left atrial dwell time: time from first cryocatheter insertion to last cryocatheter removal (mins),Change in EQ 5D
128,0.306690,-2.695985,2.256020,-0.035082,0.816136,1.063746,0.380852,-0.172024,-0.478756,-0.179739,...,-0.169332,-0.541701,0.017336,-0.584491,-0.081233,-1.452025,-0.004267,0.034946,-0.860967,0.401817
154,-0.764668,-1.718153,-1.638789,-0.353458,0.816136,1.063746,-0.311250,0.366003,-0.478756,-0.179739,...,-0.169332,-0.746231,0.017336,0.191863,0.230314,-0.191331,0.880168,0.493780,-0.225780,1.065214
276,0.407181,-3.076736,-0.143123,-0.014944,-1.765314,-1.146417,0.284545,-0.172024,-0.478756,-0.179739,...,-0.169332,1.668595,0.017336,0.778259,0.230314,-0.191331,-0.004267,-1.140601,-0.063261,0.064858
164,-1.277493,0.192692,-0.868812,0.091638,-1.765314,-1.146417,-0.515793,0.366003,-0.478756,-0.179739,...,-0.169332,1.230503,1.482320,-0.197069,0.230314,0.322677,-0.004267,0.649173,0.015393,0.064858
56,-0.434878,-1.450036,0.353945,-1.285939,-0.565279,-1.146417,0.126799,-0.172024,0.494720,-0.320770,...,-0.169332,0.085402,0.017336,-0.311874,0.230314,-0.662338,-0.004267,-0.698509,-1.285353,0.187132


In [10]:
# ╔═╡ Cell 4c – preview string-cast → impute → one-hot on categorical features ═══════╗
from sklearn.pipeline          import Pipeline
from sklearn.preprocessing     import FunctionTransformer, OneHotEncoder
from sklearn.impute            import SimpleImputer
import pandas as pd

# 1) Re-define categorical_pipe to cast → impute → one-hot
categorical_pipe = Pipeline([
    # cast every value to string
    ("to_str",   FunctionTransformer(lambda X: X.astype(str), validate=False)),
    # fill missing with literal "Missing"
    ("impute",   SimpleImputer(strategy="constant", fill_value="Missing")),
    # one-hot encode the strings
    ("onehot",   OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

# 2. Grab the raw cat columns (with NaNs)
cat_train = X_train[cat].copy()
print("🔴 Missing before imputation (categorical features):")
print(cat_train.isna().sum())

# 3. Fit & transform via the updated pipeline
cat_ohe_array = categorical_pipe.fit_transform(cat_train)

# 4. Turn back into a DataFrame
cat_ohe_cols = categorical_pipe.named_steps["onehot"].get_feature_names_out(cat)
cat_imputed  = pd.DataFrame(cat_ohe_array, columns=cat_ohe_cols, index=cat_train.index)

print("\n🟢 Missing after to_str → SimpleImputer → OneHotEncoder:")
print(cat_imputed.isna().sum().sum(), "total NaNs remaining")

print("\n🖨️ Sample of the imputed one-hot matrix:")
display(cat_imputed.iloc[:, :10].head())


🔴 Missing before imputation (categorical features):
Sex (F=1, M=0)                                                          0
Hypertension                                                            0
Diabetes                                                                0
HF                                                                      0
CAD                                                                     0
stroke                                                                  0
History of TIA                                                          0
Subject taking Class I or III AAD at baseline (1=Yes, 0=No)             5
LSPV isolated                                                           1
LIPV PV isolated                                                        2
RSPV Isolated                                                           2
RIPV isolated                                                           2
Were all targeted PVs isolated (Investigator)?              

,"Sex (F=1, M=0)_0","Sex (F=1, M=0)_1",Hypertension_0,Hypertension_1,Diabetes_0,Diabetes_1,HF_0,HF_1,CAD_0,CAD_1
128,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0
154,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0
276,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
164,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
56,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0


In [11]:
# ╔═╡ Cell 4d – inspect full preprocessed feature space ─══════════════╗
from sklearn.compose import ColumnTransformer

# 1) Re‐build the ColumnTransformer as in Cell 4
ct = ColumnTransformer([
    ("num", numeric_pipe, cont),
    ("cat", categorical_pipe, cat)
], remainder="drop")

# 2) Fit & transform your training set
X_tr_full = ct.fit_transform(X_train)

# 3) Manually assemble the feature names:
#    - numeric features keep their original names
num_features = cont

#    - categorical features come from the OneHotEncoder inside categorical_pipe
cat_ohe_names = categorical_pipe.named_steps["onehot"] \
                     .get_feature_names_out(cat)

#    - combine
feat_names = list(num_features) + list(cat_ohe_names)

# 4) Report
print(f"➡️  Combined train matrix: {X_tr_full.shape[0]} rows × {X_tr_full.shape[1]} cols")
print(f"📦  Total features = {len(feat_names)}")
print("🔍  Example feature names:", feat_names[:10], "…", feat_names[-10:])

# 5) (optional) display the first few rows in a DataFrame
import pandas as pd
df_feats = pd.DataFrame(X_tr_full, columns=feat_names, index=X_train.index)
df_feats.head()


➡️  Combined train matrix: 179 rows × 75 cols
📦  Total features = 75
🔍  Example feature names: ['age', 'BMI', 'Baseline LVEF', 'Baseline LAD', 'CHA2DS2VASc score', 'CHAD2 score', 'AF_time_procedure', 'Total no of ablation application number', 'Mobility question (Baseline)', 'Self-care question (Baseline)'] … ['Non-PVI ablation performed_1', 'Was subject taking Class I or Class III AAD at procedure discharge?_0', 'Was subject taking Class I or Class III AAD at procedure discharge?_1', 'Mapping/navigational tools: Intracardiac echocardiography (ICE)_0', 'Mapping/navigational tools: Intracardiac echocardiography (ICE)_1', 'Pre Procedural CT_0.0', 'Pre Procedural CT_1.0', 'Pre Procedural CT_nan', 'Ensite 3D mapping_0.0', 'Ensite 3D mapping_1.0']


,age,BMI,Baseline LVEF,Baseline LAD,CHA2DS2VASc score,CHAD2 score,AF_time_procedure,Total no of ablation application number,Mobility question (Baseline),Self-care question (Baseline),...,Non-PVI ablation performed_1,Was subject taking Class I or Class III AAD at procedure discharge?_0,Was subject taking Class I or Class III AAD at procedure discharge?_1,Mapping/navigational tools: Intracardiac echocardiography (ICE)_0,Mapping/navigational tools: Intracardiac echocardiography (ICE)_1,Pre Procedural CT_0.0,Pre Procedural CT_1.0,Pre Procedural CT_nan,Ensite 3D mapping_0.0,Ensite 3D mapping_1.0
128,0.306690,-2.695985,2.256020,-0.035082,0.816136,1.063746,0.380852,-0.172024,-0.478756,-0.179739,...,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
154,-0.764668,-1.718153,-1.638789,-0.353458,0.816136,1.063746,-0.311250,0.366003,-0.478756,-0.179739,...,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
276,0.407181,-3.076736,-0.143123,-0.014944,-1.765314,-1.146417,0.284545,-0.172024,-0.478756,-0.179739,...,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
164,-1.277493,0.192692,-0.868812,0.091638,-1.765314,-1.146417,-0.515793,0.366003,-0.478756,-0.179739,...,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
56,-0.434878,-1.450036,0.353945,-1.285939,-0.565279,-1.146417,0.126799,-0.172024,0.494720,-0.320770,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0


In [12]:
# ------------- 4.3 Define candidate models ------------------------------
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_pipe, cont),
        ("cat", categorical_pipe, cat)
    ],
    remainder="drop"   # or "passthrough" if you have other cols
)

In [13]:
# ╔═╡ Cell 4.4 – helper for CatBoost (just fill categoricals) ═════════════╗
def prepare_for_catboost(
    X: pd.DataFrame,
    cat_cols: list[str]
) -> pd.DataFrame:
    """
    Copy X, replace any NaN in the specified categorical columns
    with a literal 'Missing' string (so CatBoost sees it as a valid category).
    Leave all other columns untouched (CatBoost will handle numeric NaNs if any).
    """
    X_cb = X.copy()
    for c in cat_cols:
        X_cb[c] = X_cb[c].fillna("Missing").astype(str)
    return X_cb


In [14]:
# ╔═╡ Cell 4.6 – Faster tune of ExtraTrees via RandomizedSearchCV with SMOTE ═╗
from imblearn.pipeline      import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble       import ExtraTreesClassifier
import numpy as np

# 1) Pipeline: impute/encode → SMOTE → ExtraTrees
et_pipe = ImbPipeline([
    ("prep", preprocessor),
    ("smote", SMOTE(random_state=RANDOM_STATE)),
    ("clf", ExtraTreesClassifier(
        random_state=RANDOM_STATE,
        class_weight="balanced",
        n_jobs=-1          # speed up individual fits
    ))
])

# 2) Narrower/random search space
param_dist = {
    "clf__n_estimators":     [100, 300, 500, 800],
    "clf__max_depth":        [None, 10, 20],
    "clf__max_features":     ["sqrt", "log2"],
    "clf__min_samples_split":[2, 5],
    "clf__min_samples_leaf": [1, 2]
}

# 3) RandomizedSearchCV – just 30 draws instead of 720 fits
et_rand = RandomizedSearchCV(
    et_pipe,
    param_distributions=param_dist,
    n_iter=30,             # only 30 random combinations
    scoring="roc_auc",
    cv=3,
    n_jobs=-1,
    verbose=1,
    random_state=RANDOM_STATE,
    refit=True
)

# 4) Fit
et_rand.fit(X_train, y_train)

print("✅ Best ExtraTrees params:", et_rand.best_params_)
print("✅ Best CV AUC:         ", et_rand.best_score_)

# 5) Save the params
best_et_params = {
    "n_estimators":     et_rand.best_params_["clf__n_estimators"],
    "max_depth":        et_rand.best_params_["clf__max_depth"],
    "max_features":     et_rand.best_params_["clf__max_features"],
    "min_samples_split":et_rand.best_params_["clf__min_samples_split"],
    "min_samples_leaf": et_rand.best_params_["clf__min_samples_leaf"]
}

best_et_n      = best_et_params["n_estimators"]
best_et_depth  = best_et_params["max_depth"]
best_et_feats  = best_et_params["max_features"]
best_et_split  = best_et_params["min_samples_split"]
best_et_leaf   = best_et_params["min_samples_leaf"]

Fitting 3 folds for each of 30 candidates, totalling 90 fits
✅ Best ExtraTrees params: {'clf__n_estimators': 100, 'clf__min_samples_split': 5, 'clf__min_samples_leaf': 2, 'clf__max_features': 'sqrt', 'clf__max_depth': None}
✅ Best CV AUC:          0.6603707139421425


In [15]:
# ╔═╡ Cell 4.8 – Randomized tune for HistGB with SMOTE ─═════════════════╗
from imblearn.pipeline      import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
from sklearn.ensemble       import HistGradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV

# 1) Pipeline: preprocess → SMOTE → HistGB
hgb_pipe = ImbPipeline([
    ("prep", preprocessor),
    ("smote", SMOTE(random_state=RANDOM_STATE)),
    ("clf", HistGradientBoostingClassifier(
        loss="log_loss",
        early_stopping="auto",
        validation_fraction=0.1,
        n_iter_no_change=20,
        random_state=RANDOM_STATE
    ))
])

# 2) A richer space
param_dist = {
    "clf__learning_rate":      [0.005, 0.01, 0.03, 0.05, 0.1],
    "clf__max_iter":           [200, 500, 1000, 1500],
    "clf__max_depth":          [None, 3, 5, 7],
    "clf__min_samples_leaf":   [20, 50, 100],
    "clf__l2_regularization":  [0.0, 0.1, 1.0, 5.0],
    "clf__max_leaf_nodes":     [15, 31, 63]
}

# 3) RandomizedSearchCV – 30 draws, 3-fold CV
hgb_rand = RandomizedSearchCV(
    hgb_pipe,
    param_distributions=param_dist,
    n_iter=30,
    scoring="roc_auc",
    cv=3,
    n_jobs=-1,
    verbose=1,
    random_state=RANDOM_STATE,
    refit=True
)

# 4) Fit it
hgb_rand.fit(X_train, y_train)

# 5) Capture the winner
best_hgb_params = {
    k.replace("clf__", ""): v
    for k, v in hgb_rand.best_params_.items()
}
print("✅ Best HistGB params:", best_hgb_params,
      "→ CV AUC:", hgb_rand.best_score_)


Fitting 3 folds for each of 30 candidates, totalling 90 fits
✅ Best HistGB params: {'min_samples_leaf': 50, 'max_leaf_nodes': 15, 'max_iter': 200, 'max_depth': 5, 'learning_rate': 0.01, 'l2_regularization': 5.0} → CV AUC: 0.702182861781076


In [16]:
# ╔═╡ Cell 5 – Build models dict with SMOTE in every pipeline ─════════╗
from imblearn.pipeline       import Pipeline as ImbPipeline
from imblearn.over_sampling  import SMOTE
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import numpy as np

models = {
    "LogReg": RandomizedSearchCV(
        ImbPipeline([
            ("prep",  preprocessor),
            ("smote", SMOTE(random_state=RANDOM_STATE)),
            ("clf",   LogisticRegression(
                          solver="saga",
                          penalty="elasticnet",
                          l1_ratio=0.5,
                          class_weight="balanced",
                          max_iter=5000,
                          random_state=RANDOM_STATE
                      ))
        ]),
        param_distributions={
            "clf__C":        [1e-3, 1e-2, 1e-1, 1, 10, 100],
            "clf__l1_ratio":[0.0, 0.25, 0.5, 0.75, 1.0]
        },
        n_iter=20,
        scoring="roc_auc",
        cv=5,
        n_jobs=-1,
        verbose=1,
        random_state=RANDOM_STATE,
        refit=True
    ),

    "SVM": RandomizedSearchCV(
        ImbPipeline([
            ("prep",  preprocessor),
            ("smote", SMOTE(random_state=RANDOM_STATE)),
            ("clf",   SVC(
                          probability=True,
                          kernel="rbf",
                          class_weight="balanced",
                          random_state=RANDOM_STATE
                      ))
        ]),
        param_distributions={
            "clf__C":     np.logspace(-3, 3, 20),
            "clf__gamma": np.logspace(-4, 0, 20)
        },
        n_iter=30,
        scoring="roc_auc",
        cv=5,
        n_jobs=-1,
        verbose=1,
        random_state=RANDOM_STATE,
        refit=True
    ),

    "NaiveBayes": GridSearchCV(
        ImbPipeline([
            ("prep",  preprocessor),
            ("smote", SMOTE(random_state=RANDOM_STATE)),
            ("clf",   GaussianNB())
        ]),
        param_grid={"clf__var_smoothing": np.logspace(-12, -6, 6)},
        scoring="roc_auc",
        cv=5,
        n_jobs=-1,
        verbose=1,
        refit=True
    ),

    "ExtraTrees": ImbPipeline([
        ("prep",  preprocessor),
        ("smote", SMOTE(random_state=RANDOM_STATE)),
        ("clf",   ExtraTreesClassifier(
                      n_estimators=     best_et_n,
                      max_depth=        best_et_depth,
                      max_features=     best_et_feats,
                      min_samples_split=best_et_split,
                      min_samples_leaf= best_et_leaf,
                      random_state=     RANDOM_STATE,
                      class_weight=     "balanced",
                      n_jobs=-1
                  ))
    ]),

    "HistGB": ImbPipeline([
        ("prep",  preprocessor),
        ("smote", SMOTE(random_state=RANDOM_STATE)),
        ("clf",   HistGradientBoostingClassifier(
                      loss=               "log_loss",
                      learning_rate=      best_hgb_params["learning_rate"],
                      max_iter=           best_hgb_params["max_iter"],
                      max_depth=          best_hgb_params["max_depth"],
                      min_samples_leaf=   best_hgb_params["min_samples_leaf"],
                      l2_regularization=  best_hgb_params["l2_regularization"],
                      max_leaf_nodes=     best_hgb_params["max_leaf_nodes"],
                      early_stopping=     "auto",
                      validation_fraction=0.1,
                      n_iter_no_change=   20,
                      random_state=       RANDOM_STATE
                  ))
    ])
}

# If you still need the cat_features_idx for SHAP or CatBoost elsewhere:
cat_features_idx = [X_train.columns.get_loc(c) for c in cat]


In [17]:
# ╔═╡ Cell 5.1 – Full‐train smoke test of all pipelines ═════════════════╗
from sklearn.metrics import roc_auc_score

print("=== Pipeline structure ===")
for name, pipe in models.items():
    print(f"{name}:")
    if hasattr(pipe, "steps"):
        print("  Steps:", [step for step, _ in pipe.steps])
    else:
        print("  (not a pipeline!)", type(pipe).__name__)
    print()

print("=== Smoke test (fit on full X_train, score on X_test) ===")
for name, pipe in models.items():
    try:
        pipe.fit(X_train, y_train)
        probs = pipe.predict_proba(X_test)[:, 1]
        auc = roc_auc_score(y_test, probs)
        print(f"{name}: ✔ fitted & predict_proba OK → AUC on hold-out = {auc:.3f}")
    except Exception as e:
        print(f"{name}: ✖ ERROR → {e}")



=== Pipeline structure ===
LogReg:
  (not a pipeline!) RandomizedSearchCV

SVM:
  (not a pipeline!) RandomizedSearchCV

NaiveBayes:
  (not a pipeline!) GridSearchCV

ExtraTrees:
  Steps: ['prep', 'smote', 'clf']

HistGB:
  Steps: ['prep', 'smote', 'clf']

=== Smoke test (fit on full X_train, score on X_test) ===
Fitting 5 folds for each of 20 candidates, totalling 100 fits
LogReg: ✔ fitted & predict_proba OK → AUC on hold-out = 0.691
Fitting 5 folds for each of 30 candidates, totalling 150 fits
SVM: ✔ fitted & predict_proba OK → AUC on hold-out = 0.500
Fitting 5 folds for each of 6 candidates, totalling 30 fits
NaiveBayes: ✔ fitted & predict_proba OK → AUC on hold-out = 0.642
ExtraTrees: ✔ fitted & predict_proba OK → AUC on hold-out = 0.565
HistGB: ✔ fitted & predict_proba OK → AUC on hold-out = 0.415


In [18]:
# ╔═╡ Cell 5.2 – Tune CatBoost on raw data (no SMOTE) with repeated stratified CV ══════════╗
from catboost import CatBoostClassifier
from sklearn.model_selection import RandomizedSearchCV, RepeatedStratifiedKFold
import numpy as np

# 1) Prepare your train DataFrame for CatBoost: fill NaNs in cat‐cols with "Missing" as strings
X_train_cb = prepare_for_catboost(X_train, cat)

# 2) Define base CatBoost and richer hyperparameter space
cb = CatBoostClassifier(
    loss_function="Logloss",
    eval_metric="AUC",
    random_seed=RANDOM_STATE,
    verbose=False
)
param_dist = {
    "iterations":    [200, 500, 1000],
    "depth":         [4, 6, 8],
    "learning_rate": [0.01, 0.03, 0.05, 0.1],
    "l2_leaf_reg":   [1, 3, 5],
    "subsample":     [0.6, 0.8, 1.0],
    "border_count":  [32, 64, 128],
}

# 3) Repeated stratified K-fold for stability on imbalanced data
rskf = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=RANDOM_STATE)

# 4) RandomizedSearchCV on the raw CatBoostClassifier
rs_cb = RandomizedSearchCV(
    cb,
    param_distributions=param_dist,
    n_iter=10,
    scoring="roc_auc",
    cv=rskf,
    n_jobs=-1,
    verbose=2,
    random_state=RANDOM_STATE,
    refit=True
)

# 5) Fit—but pass `cat_features` so CatBoost knows which cols are categorical
rs_cb.fit(
    X_train_cb,
    y_train,
    cat_features=cat_features_idx
)

print("✅ Best CatBoost params:", rs_cb.best_params_)
print("✅ Best CV AUC:        ", rs_cb.best_score_)

# 6) Save the tuned CatBoostClassifier (bare, not wrapped in a pipeline)
models["CatBoost"] = rs_cb.best_estimator_

# 7) (Re)compute your categorical‐feature indices for SHAP / Cell 6
cat_features_idx = [X_train.columns.get_loc(c) for c in cat]


Fitting 15 folds for each of 10 candidates, totalling 150 fits
✅ Best CatBoost params: {'subsample': 0.8, 'learning_rate': 0.01, 'l2_leaf_reg': 5, 'iterations': 200, 'depth': 8, 'border_count': 128}
✅ Best CV AUC:         0.6433497536945811


In [19]:
# ╔═╡ Cell 5.3 – Deep hyper-tuning AdaBoost with SMOTE & repeated stratified CV ══╗
from imblearn.pipeline            import Pipeline as ImbPipeline
from imblearn.over_sampling       import SMOTE
from sklearn.tree                 import DecisionTreeClassifier
from sklearn.ensemble             import AdaBoostClassifier
from sklearn.model_selection      import RandomizedSearchCV, RepeatedStratifiedKFold
import numpy as np

# 1) Pipeline: preprocess → SMOTE → AdaBoost(with explicit base tree)
ab_pipe = ImbPipeline([
    ("prep",  preprocessor),
    ("smote", SMOTE(random_state=RANDOM_STATE)),
    ("clf",   AdaBoostClassifier(
                  estimator=DecisionTreeClassifier(
                      random_state=RANDOM_STATE
                  ),
                  random_state=RANDOM_STATE
              ))
])

# 2) A richer Randomized grid
param_dist = {
    "clf__n_estimators":                [50, 100, 200, 400, 800],
    "clf__learning_rate":               [0.01, 0.05, 0.1, 0.5, 1.0],
    "clf__algorithm":                   ["SAMME", "SAMME.R"],
    "clf__estimator__max_depth":        [1, 2, 3, 4, None],
    "clf__estimator__min_samples_split":[2, 5, 10],
    "clf__estimator__min_samples_leaf": [1, 2, 4]
}

# 3) Repeated stratified K-fold
rskf = RepeatedStratifiedKFold(
    n_splits=3,
    n_repeats=3,
    random_state=RANDOM_STATE
)

# 4) RandomizedSearchCV
ab_search = RandomizedSearchCV(
    ab_pipe,
    param_distributions=param_dist,
    n_iter=50,                # 50 random draws
    scoring="roc_auc",
    cv=rskf,
    n_jobs=-1,
    verbose=2,
    random_state=RANDOM_STATE,
    refit=True
)

# 5) Fit on your build set
ab_search.fit(X_train, y_train)

print("✅ Best AdaBoost params:", ab_search.best_params_)
print("✅ Best CV AUC:        ", ab_search.best_score_)

# 6) Store the tuned pipeline
models["AdaBoost"] = ab_search.best_estimator_


Fitting 9 folds for each of 50 candidates, totalling 450 fits
✅ Best AdaBoost params: {'clf__n_estimators': 800, 'clf__learning_rate': 1.0, 'clf__estimator__min_samples_split': 5, 'clf__estimator__min_samples_leaf': 2, 'clf__estimator__max_depth': 4, 'clf__algorithm': 'SAMME.R'}
✅ Best CV AUC:         0.6509954992097848


In [20]:
# ╔═╡ Cell 5.4 – Enhanced Randomized search → final XGBoost ═════════════╗
import xgboost as xgb
from sklearn.pipeline        import Pipeline
from sklearn.impute          import SimpleImputer
from sklearn.preprocessing   import OrdinalEncoder
from sklearn.compose         import ColumnTransformer
from sklearn.model_selection import RandomizedSearchCV, RepeatedStratifiedKFold
import numpy as np

# 1) Simple preprocessor
num_imp     = SimpleImputer(strategy="median")
cat_imp_enc = Pipeline([
    ("impute", SimpleImputer(strategy="constant", fill_value=-1)),
    ("encode", OrdinalEncoder(
         handle_unknown="use_encoded_value",
         unknown_value=-1
    ))
])
pre_xgb = ColumnTransformer([
    ("num", num_imp, cont),
    ("cat", cat_imp_enc, cat)
], remainder="drop")

# 2) Base pipeline (no early stopping in CV)
xgb_pipe = Pipeline([
    ("pre", pre_xgb),
    ("clf", xgb.XGBClassifier(
        objective="binary:logistic",
        eval_metric="auc",
        tree_method="hist",
        scale_pos_weight=(y_train==0).sum()/(y_train==1).sum(),
        random_state=RANDOM_STATE,
        use_label_encoder=False
    ))
])

# 3) Expanded random search space
param_dist = {
    "clf__n_estimators":    [100, 300, 600, 1000],
    "clf__max_depth":       [3, 5, 7, 9],
    "clf__learning_rate":   [0.001, 0.01, 0.03, 0.1],
    "clf__subsample":       [0.6, 0.8, 1.0],
    "clf__colsample_bytree":[0.6, 0.8, 1.0],
    "clf__gamma":           [0, 1, 5],
    "clf__reg_alpha":       [0, 0.1, 1],
    "clf__reg_lambda":      [1, 5, 10]
}

# 4) Use repeated stratified CV for more stable estimates
rsk = RepeatedStratifiedKFold(n_splits=5, n_repeats=2, random_state=RANDOM_STATE)

rs_xgb = RandomizedSearchCV(
    xgb_pipe,
    param_distributions=param_dist,
    n_iter=20,
    scoring="roc_auc",
    cv=rsk,
    n_jobs=-1,
    verbose=2,
    random_state=RANDOM_STATE,
    refit=True
)

# 5) Run the search
rs_xgb.fit(X_train, y_train)

print("✅ Best XGB params:", rs_xgb.best_params_)
print("✅ Best CV AUC:   ", rs_xgb.best_score_)

# 6) Build final XGB with early stopping on your temporal hold-out
best_params = {
    k.replace("clf__", ""): v
    for k, v in rs_xgb.best_params_.items()
}
xgb_final = xgb.XGBClassifier(
    **best_params,
    objective="binary:logistic",
    eval_metric="auc",
    tree_method="hist",
    scale_pos_weight=(y_train==0).sum()/(y_train==1).sum(),
    early_stopping_rounds=30,
    random_state=RANDOM_STATE,
    use_label_encoder=False
)

# 7) Fit the preprocessor & transform both build and hold-out
pre_xgb.fit(X_train)
X_tr = pre_xgb.transform(X_train)
X_te = pre_xgb.transform(X_test)

# 8) Train with early stopping on the test slice
xgb_final.fit(
    X_tr, y_train,
    eval_set=[(X_te, y_test)],
    verbose=False
)

# 9) Save the pipeline for downstream use
models["XGBoost"] = Pipeline([("pre", pre_xgb), ("clf", xgb_final)])


Fitting 10 folds for each of 20 candidates, totalling 200 fits


/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [10:51:22] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.11/dist-packages/xgboost/callback.py:386: UserWarning: [10:51:22] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  self.starting_round = model.num_boosted_rounds()


✅ Best XGB params: {'clf__subsample': 0.8, 'clf__reg_lambda': 10, 'clf__reg_alpha': 1, 'clf__n_estimators': 300, 'clf__max_depth': 3, 'clf__learning_rate': 0.03, 'clf__gamma': 0, 'clf__colsample_bytree': 1.0}
✅ Best CV AUC:    0.6483579638752053


In [21]:
# ╔═╡ Cell 6 – 80/20 StratifiedShuffleSplit ROC (mean ± std) with SMOTE (except CatBoost) ─════╗
import numpy as np
from sklearn.model_selection   import StratifiedShuffleSplit
from sklearn.metrics           import roc_curve, roc_auc_score, RocCurveDisplay
from sklearn.base              import clone
from imblearn.over_sampling     import SMOTE
from imblearn.pipeline          import Pipeline as ImbPipeline
from sklearn.pipeline           import Pipeline as SkPipeline

# 1) Build “full” pipelines for every model, but skip SMOTE for CatBoost
full_pipes = {}
for name, mdl in models.items():
    # unwrap CV wrappers if present
    pipe = mdl.best_estimator_ if hasattr(mdl, "best_estimator_") else mdl

    if isinstance(pipe, (SkPipeline, ImbPipeline)):
        # already a pipeline: leave as-is
        full_pipes[name] = pipe

    else:
        # bare estimator: wrap with preprocessor + (optional SMOTE) + clf
        steps = [("prep", preprocessor)]
        if name != "CatBoost":
            steps.append(("smote", SMOTE(random_state=RANDOM_STATE)))
        steps.append(("clf", pipe))
        full_pipes[name] = SkPipeline(steps) if name == "CatBoost" else ImbPipeline(steps)

# 2) Prepare 5×80/20 stratified splits
sss       = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=RANDOM_STATE)
mean_fpr  = np.linspace(0, 1, 200)
aucs      = {n: [] for n in full_pipes}
mean_tpr  = {n: np.zeros_like(mean_fpr) for n in full_pipes}

# 3) Loop folds, clone & fit each full pipeline
for fold, (tr_idx, val_idx) in enumerate(sss.split(X_train, y_train), start=1):
    X_tr, X_val = X_train.iloc[tr_idx], X_train.iloc[val_idx]
    y_tr, y_val = y_train.iloc[tr_idx], y_train.iloc[val_idx]
    print(f"\n--- Fold {fold} ---")
    for name, pipe in full_pipes.items():
        clf = clone(pipe)
        clf.fit(X_tr, y_tr)                       # pipeline does prep→[SMOTE]→clf
        y_prob = clf.predict_proba(X_val)[:, 1]

        fpr, tpr, _ = roc_curve(y_val, y_prob)
        auc_score   = roc_auc_score(y_val, y_prob)
        aucs[name].append(auc_score)
        mean_tpr[name] += np.interp(mean_fpr, fpr, tpr)
        print(f"{name:12s} AUC = {auc_score:.3f}")

# 4) Summarize per‐fold AUCs
print("\n=== Summary of per‐fold AUCs ===")
for name, scores in aucs.items():
    m, s = np.mean(scores), np.std(scores)
    print(f"{name:12s}: {['{:.3f}'.format(x) for x in scores]} → {m:.3f}±{s:.3f}")

# 5) Build averaged ROC displays for Cell 7
roc_displays = []
for name in full_pipes:
    mean_tpr[name] /= sss.get_n_splits()
    mean_tpr[name][0], mean_tpr[name][-1] = 0.0, 1.0
    m, s = np.mean(aucs[name]), np.std(aucs[name])
    roc_displays.append(
        RocCurveDisplay(
            fpr=mean_fpr,
            tpr=mean_tpr[name],
            estimator_name=f"{name} (AUC={m:.3f}±{s:.3f})"
        )
    )



--- Fold 1 ---
LogReg       AUC = 0.576
SVM          AUC = 0.424
NaiveBayes   AUC = 0.416
ExtraTrees   AUC = 0.562
HistGB       AUC = 0.626
CatBoost     AUC = 0.660
AdaBoost     AUC = 0.709


/usr/local/lib/python3.11/dist-packages/xgboost/callback.py:386: UserWarning: [10:51:31] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  self.starting_round = model.num_boosted_rounds()


ValueError: Must have at least 1 validation dataset for early stopping.

In [ ]:
# ╔═╡ Cell 7 – Plot combined ROC figure ─════════════════════════════╗
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 6))
colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]

for idx, disp in enumerate(roc_displays):
    disp.plot(ax=plt.gca(), alpha=0.85, linewidth=2, color=colors[idx % len(colors)])

plt.plot([0, 1], [0, 1], "--", color="grey", linewidth=1)
plt.title("ROC – Recurrence_2yr")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend(loc="lower right")
plt.tight_layout()
plt.show()


In [ ]:
# ╔═╡ Cell 7b – Bootstrap AUC 95% CI ─────────────────────────────────╗
import numpy as np
from sklearn.metrics    import roc_auc_score

# 1) ground truth
y_true = y_test.reset_index(drop=True)

# 2) pipeline handles preprocessing internally
y_prob = models["XGBoost"].predict_proba(
    X_test.reset_index(drop=True)
)[:, 1]

# 3) bootstrap
n_bootstraps = 1000
rng = np.random.RandomState(42)
scores = []

for _ in range(n_bootstraps):
    idx = rng.randint(0, len(y_true), len(y_true))
    if len(np.unique(y_true.iloc[idx])) < 2:
        continue
    scores.append(roc_auc_score(y_true.iloc[idx], y_prob[idx]))

ci_lower, ci_upper = np.percentile(scores, [2.5, 97.5])
auc_mean = roc_auc_score(y_true, y_prob)

print(f"AUC (test)         = {auc_mean:.3f}")
print(f"95% CI (bootstrap)= [{ci_lower:.3f}, {ci_upper:.3f}]")


In [ ]:
# ╔═╡ Cell 8 – Bar chart of mean ShuffleSplit AUCs with ±std ─════════╗
import numpy as np

# 1. Compute mean & std per model
mean_aucs = {name: np.mean(vals) for name, vals in aucs.items()}
std_aucs  = {name: np.std(vals)  for name, vals in aucs.items()}

# 2. Sort by mean AUC
sorted_items = sorted(mean_aucs.items(), key=lambda kv: kv[1])
labels, means = zip(*sorted_items)
stds = [std_aucs[name] for name in labels]

# 3. Plot bars with error bars
plt.figure(figsize=(10, 4))
bars = plt.bar(labels, means, yerr=stds, capsize=4)
plt.ylim(0, 1)
plt.ylabel("Mean AUC")
plt.title("Mean AUC ± Std – 80/20 ShuffleSplit (5 runs)")
plt.xticks(rotation=30, ha="right")

# 4. Annotate each bar with mean±std
for bar, m, s in zip(bars, means, stds):
    text = f"{m:.3f}±{s:.3f}"
    plt.annotate(
        text,
        xy=(bar.get_x() + bar.get_width() / 2, bar.get_height()),
        xytext=(0, 3), textcoords="offset points",
        ha="center", va="bottom", fontsize=8
    )

plt.tight_layout()
plt.show()


In [ ]:
# ╔═╡ Cell 9 – Quick summary table with mean ± std AUC ─════════════╗
import numpy as np

# Build a list of records
records = []
for name, vals in aucs.items():
    records.append({
        "Model":    name,
        "Mean AUC": np.mean(vals),
        "Std AUC":  np.std(vals)
    })

# Create DataFrame and sort
summary = (
    pd.DataFrame(records)
      .sort_values("Mean AUC", ascending=False)
      .reset_index(drop=True)
)

# Plain‐text print
print(summary.to_string(index=False, float_format="%.3f"))


In [ ]:
# ╔═╡ Cell 10.1 – Pipeline + sigmoid calibration ─────────────────────╗
from sklearn.pipeline            import Pipeline
from sklearn.calibration        import CalibratedClassifierCV
from xgboost                    import XGBClassifier

# 1) Build an XGB pipeline WITHOUT early stopping
#    This uses the same hyperparameters you found in Cell 5.4 (best_params).
xgb_pipe_uncal = Pipeline([
    ("prep", pre_xgb),  # your simple median‐imputer + ordinal‐encoder ColumnTransformer
    ("clf",  XGBClassifier(
        **best_params,               # from your RandomizedSearchCV in Cell 5.4
        objective="binary:logistic",
        eval_metric="auc",
        tree_method="hist",
        scale_pos_weight=(y_train==0).sum()/(y_train==1).sum(),
        random_state=RANDOM_STATE
        # <— NO early_stopping_rounds here!
    ))
])

# 2) Wrap in a Platt‐scaler (sigmoid) using 5‐fold CV on the full training set
calibrator = CalibratedClassifierCV(
    estimator=xgb_pipe_uncal,
    method="sigmoid",  # Platt scaling
    cv=5
)

# 3) Fit on your entire training set
calibrator.fit(X_train, y_train)

# 4) Store back into your models dict
models["XGBoost_Calibrated"] = calibrator

print("✅ Calibrated XGBoost fitted with sigmoid scaling (5‐fold CV).")


In [ ]:
# ╔═╡ Cell 10.2 – Calibration curve + Brier score ─────────────────══════╗
from sklearn.calibration import calibration_curve
from sklearn.metrics      import brier_score_loss
import matplotlib.pyplot as plt

# 1) Raw probabilities (before calibration) on the raw X_test
y_prob_raw = models["XGBoost"].predict_proba(X_test)[:, 1]

# 2) Calibrated probabilities (same raw X_test)
y_prob_cal = models["XGBoost_Calibrated"].predict_proba(X_test)[:, 1]

# 3) Compute calibration curves (fraction of positives vs. mean predicted)
frac_pos_raw, mean_pred_raw = calibration_curve(y_test, y_prob_raw, n_bins=10)
frac_pos_cal, mean_pred_cal = calibration_curve(y_test, y_prob_cal, n_bins=10)

plt.figure(figsize=(6, 6))
plt.plot(mean_pred_raw, frac_pos_raw, "s-", label="XGBoost (raw)")
plt.plot(mean_pred_cal, frac_pos_cal, "o-", label="XGBoost (calibrated)")
plt.plot([0, 1], [0, 1], "k--", label="Perfectly calibrated")
plt.xlabel("Mean predicted probability")
plt.ylabel("Fraction of positives")
plt.title("Calibration curve – XGBoost")
plt.legend(loc="lower right")
plt.tight_layout()
plt.show()

# 4) Brier scores for raw vs. calibrated
print("Brier score (raw)     :", brier_score_loss(y_test, y_prob_raw))
print("Brier score (calibr.) :", brier_score_loss(y_test, y_prob_cal))


In [ ]:
# ╔═╡ Cell 10.3 – Precision–Recall & threshold‐tuning ════════════════╗
from sklearn.metrics import precision_recall_curve, f1_score

y_prob = models["XGBoost"].predict_proba(X_test)[:,1]
precision, recall, thresholds = precision_recall_curve(y_test, y_prob)

plt.figure(figsize=(6,4))
plt.plot(recall, precision, label="XGBoost PR-curve")
plt.xlabel("Recall"); plt.ylabel("Precision")
plt.title("Precision–Recall curve")
plt.legend()
plt.tight_layout()
plt.show()

# F1 vs threshold
f1s = [f1_score(y_test, y_prob>=t) for t in thresholds]
opt_idx = np.argmax(f1s)
print(f"Best F1={f1s[opt_idx]:.3f} at threshold={thresholds[opt_idx]:.2f}")


In [ ]:
# ╔═╡ Cell 10.4 – SHAP summary (model-agnostic) ─────────────────══════╗
import shap
import matplotlib.pyplot as plt

# 1) Make a simple function that takes a DataFrame X and returns P(y=1)
def model_proba(df):
    return models["XGBoost"].predict_proba(df)[:, 1]

# 2) Build the explainer on your training DataFrame
explainer = shap.Explainer(model_proba, X_train)

# 3) Compute SHAP values on your held-out test set
shap_values = explainer(X_test)

# 4) Beeswarm (dot) summary
plt.figure(figsize=(10, 6))
shap.plots.beeswarm(shap_values, max_display=20, show=False)
plt.title("SHAP Beeswarm – XGBoost (Recurrence_2yr)")
plt.tight_layout()
plt.show()

# 5) Bar plot of mean(|SHAP|)
plt.figure(figsize=(6, 8))
shap.plots.bar(shap_values, max_display=20, show=False)
plt.title("mean(|SHAP|) – XGBoost")
plt.tight_layout()
plt.show()


In [ ]:
# ╔═╡ Cell 10.5 – Partial dependence for top features ═══════════════╗
import numpy as np
from sklearn.inspection import PartialDependenceDisplay
import matplotlib.pyplot as plt

# 1) Extract the raw SHAP values array from the Explanation
#    shap_values was created in Cell 10 via `explainer(X_test)`
vals = shap_values.values
# If it’s a 3‐D array (n_samples x 2 classes x n_features),
# pick the positive‐class slice:
if vals.ndim == 3:
    vals = vals[:, 1, :]

# 2) Compute mean(|SHAP|) per feature and grab top-3 indices
importances = np.abs(vals).mean(axis=0)
top_idx    = np.argsort(importances)[::-1][:3]
top_feats  = [X_test.columns[i] for i in top_idx]

print("Top 3 features by |SHAP|:", top_feats)

# 3) Plot partial dependence for those top 3
fig, axes = plt.subplots(1, len(top_feats), figsize=(4*len(top_feats), 4))
PartialDependenceDisplay.from_estimator(
    models["XGBoost"],
    X_test,
    features=top_feats,
    ax=axes
)
plt.tight_layout()
plt.show()


In [ ]:
# ╔═╡ Cell 10.6 – Permutation importance ══════════════════════════╗
from sklearn.inspection import permutation_importance

r = permutation_importance(
    models["XGBoost"], X_test, y_test,
    scoring="roc_auc", n_repeats=10, random_state=RANDOM_STATE
)
perm_df = pd.Series(r.importances_mean, index=X_test.columns).sort_values(ascending=False)
perm_df.head(10).plot.barh()
plt.xlabel("Mean decrease in AUC")
plt.title("Permutation importance – XGBoost")
plt.tight_layout()
plt.show()

In [ ]:
# ╔═╡ Cell 10.7 – Manual Decision Curve Analysis (Net Benefit) ─════════════╗
import numpy as np
import matplotlib.pyplot as plt

# 1) True labels & predicted probabilities (from your best model)
y_true = y_test.values        # shape (n_samples,)
y_prob = models["XGBoost"].predict_proba(X_test)[:,1]

N      = len(y_true)
preval = y_true.mean()        # event rate

# 2) Define a grid of thresholds
thresholds = np.linspace(0.01, 0.99, 99)

# 3) Compute net benefit for each threshold
nb_model = []
nb_all   = []
nb_none  = np.zeros_like(thresholds)

for pt in thresholds:
    preds = (y_prob >= pt).astype(int)
    TP    = ((preds == 1) & (y_true == 1)).sum()
    FP    = ((preds == 1) & (y_true == 0)).sum()
    # NB = TP/N - FP/N * (pt/(1-pt))
    nb    = TP/N - FP/N * (pt/(1-pt))
    nb_model.append(nb)
    nb_all.append(preval - (1 - preval)*(pt/(1-pt)))

# 4) Plot
plt.figure(figsize=(7,5))
plt.plot(thresholds, nb_model, label="XGBoost")
plt.plot(thresholds, nb_all,   label="Treat All",    linestyle="--")
plt.plot(thresholds, nb_none,  label="Treat None",   linestyle=":")
plt.xlabel("Decision Threshold")
plt.ylabel("Net Benefit")
plt.title("Decision Curve Analysis – Recurrence_2yr")
plt.legend(loc="best")
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
# ╔═╡ Cell 10.8 – Final evaluation on future hold‐out ═══════════════════╗
from sklearn.metrics       import (
    roc_auc_score,
    confusion_matrix,
    classification_report,
    RocCurveDisplay,
    brier_score_loss,
    roc_curve
)
from sklearn.calibration   import calibration_curve
import matplotlib.pyplot   as plt
import numpy              as np

# 1) Pick your final model
best_model    = models["XGBoost"]    # or "XGBoost_Calibrated"
X_val, y_val  = X_test, y_test

# 2) Predicted probabilities & hard labels
y_val_proba = best_model.predict_proba(X_val)[:, 1]
y_val_pred  = best_model.predict(X_val)

# 3) Print metrics
print("Final AUC:            ", roc_auc_score(y_val, y_val_proba))
print("Confusion matrix (0.5):\n", confusion_matrix(y_val, y_val_pred))
print("\nClassification report:\n", classification_report(y_val, y_val_pred))

# 4) ROC curve
RocCurveDisplay.from_predictions(y_val, y_val_proba)
plt.title("Final ROC on hold‐out")
plt.tight_layout()
plt.show()

# 5) Calibration curve & Brier score
frac_pos, mean_pred = calibration_curve(y_val, y_val_proba, n_bins=10)
plt.figure(figsize=(6, 4))
plt.plot(mean_pred, frac_pos, "o-", label="XGB hold‐out")
plt.plot([0, 1], [0, 1], "k--",  label="Perfectly calibrated")
plt.xlabel("Mean predicted probability")
plt.ylabel("Fraction of positives")
plt.title("Calibration curve – hold‐out")
plt.legend()
plt.tight_layout()
plt.show()

print("Final Brier score:", brier_score_loss(y_val, y_val_proba))

# 6) Optimal threshold via Youden’s J
fpr, tpr, thresh = roc_curve(y_val, y_val_proba)
j_scores         = tpr - fpr
best_thresh      = thresh[np.argmax(j_scores)]
print(f"Optimal threshold (max Youden's J): {best_thresh:.3f}")


In [ ]:
# ╔═╡ Cell 10.9 – Bootstrap‐Calibrated XGBoost on future hold‐out ═════════╗
import numpy as np
import matplotlib.pyplot as plt

from sklearn.utils           import resample
from sklearn.base            import clone
from sklearn.metrics         import (
    roc_auc_score, roc_curve,
    confusion_matrix,
    classification_report,
    RocCurveDisplay,
    brier_score_loss
)
from sklearn.calibration     import CalibratedClassifierCV

# 1) Number of bootstrap replicates
n_boot = 20

# 2) Prepare array to hold each replicate’s hold‐out probs
boot_preds = np.zeros((len(X_test), n_boot))

# 3) Loop: bootstrap sample → clone uncalibrated pipeline → calibrate → predict
for i in range(n_boot):
    # a) draw a bootstrap sample from your build set
    Xb, yb = resample(X_train, y_train, random_state=RANDOM_STATE + i)
    # b) clone the UNCALIBRATED XGB pipeline you built in Cell 10.1
    xgb_pipe = clone(xgb_pipe_uncal)  # <-- no early stopping here!
    # c) wrap in a Platt‐scaler (sigmoid) calibrator
    calib = CalibratedClassifierCV(
        estimator=xgb_pipe,
        method="sigmoid",
        cv=3
    )
    # d) fit on the bootstrap draw
    calib.fit(Xb, yb)
    # e) predict probabilities on the true 20% hold‐out
    boot_preds[:, i] = calib.predict_proba(X_test)[:, 1]

# 4) average the bootstrap‐calibrated probabilities
y_mean = boot_preds.mean(axis=1)
y_pred = (y_mean >= 0.5).astype(int)

# 5) Discrimination & classification metrics
print("Bootstrap‐Calibrated XGBoost AUC:   ",
      roc_auc_score(y_test, y_mean))
print("Confusion matrix @0.5:\n",
      confusion_matrix(y_test, y_pred))
print("\nClassification report:\n",
      classification_report(y_test, y_pred))

# 6) Plot ROC
RocCurveDisplay.from_predictions(y_test, y_mean)
plt.title("Bootstrap‐Calibrated XGBoost – ROC on hold‐out")
plt.tight_layout()
plt.show()

# 7) Calibration curve + Brier score
from sklearn.calibration import calibration_curve
frac_pos, mean_pred = calibration_curve(y_test, y_mean, n_bins=10)
plt.figure(figsize=(6, 4))
plt.plot(mean_pred, frac_pos, "o-", label="Boot‐Calibrated XGB")
plt.plot([0, 1], [0, 1], "k--", label="Perfectly calibrated")
plt.xlabel("Mean predicted probability")
plt.ylabel("Fraction of positives")
plt.title("Calibration curve – hold‐out")
plt.legend()
plt.tight_layout()
plt.show()

print("Bootstrap‐Calibrated XGB Brier score:",
      brier_score_loss(y_test, y_mean))

# 8) Optimal threshold via Youden’s J
fpr, tpr, thresh = roc_curve(y_test, y_mean)
j_scores       = tpr - fpr
best_thresh    = thresh[np.argmax(j_scores)]
print(f"Optimal threshold (max Youden’s J): {best_thresh:.3f}")


In [ ]:
# ╔═╡ Cell 11.0 – SHAP summary for CatBoost (Recurrence_2yr) ═════════════╗
import shap
import matplotlib.pyplot as plt
import numpy as np
from sklearn.pipeline import Pipeline as SkPipeline
from imblearn.pipeline import Pipeline as ImbPipeline

# 1) Unpack your CatBoost model + preprocessor
cb_entry = models["CatBoost"]
if isinstance(cb_entry, (SkPipeline, ImbPipeline)):
    prep     = cb_entry.named_steps["prep"]
    cb_model = cb_entry.named_steps["clf"]
else:
    prep     = preprocessor      # fallback to your ColumnTransformer
    cb_model = cb_entry          # bare CatBoostClassifier

# 2) Transform to the numeric matrix CatBoost was trained on
X_train_pre = prep.transform(X_train)
X_test_pre  = prep.transform(X_test)

# 3) Build a TreeExplainer
explainer_cb = shap.TreeExplainer(cb_model, X_train_pre)

# 4) Compute SHAP values on your hold-out
shap_values_cb = explainer_cb.shap_values(X_test_pre)
print("🔎 SHAP values shape:", np.array(shap_values_cb).shape)

# 5) Reconstruct feature names: continuous + one-hot cats
ohe            = prep.named_transformers_["cat"].named_steps["onehot"]
cat_ohe_names  = ohe.get_feature_names_out(cat)
feat_names     = cont + list(cat_ohe_names)

# 6) Beeswarm (dot) summary plot
plt.figure(figsize=(10, 6))
shap.summary_plot(
    shap_values_cb,
    X_test_pre,
    feature_names=feat_names,
    max_display=20,
    show=False
)
plt.title("SHAP Beeswarm – CatBoost (Recurrence_2yr)")
plt.tight_layout()
plt.show()

# 7) Bar plot of mean(|SHAP|)
plt.figure(figsize=(6, 8))
shap.summary_plot(
    shap_values_cb,
    X_test_pre,
    feature_names=feat_names,
    plot_type="bar",
    max_display=20,
    show=False
)
plt.title("mean(|SHAP|) – CatBoost")
plt.tight_layout()
plt.show()


In [ ]:
# ╔═╡ Cell 11.1 – Partial dependence (top 3 preprocessed features) ═════════╗
from sklearn.inspection import PartialDependenceDisplay
import numpy as np
import matplotlib.pyplot as plt

# 1) Unpack your CatBoost pipeline or raw model
cb_entry = models["CatBoost"]
if isinstance(cb_entry, (SkPipeline, ImbPipeline)):
    prep     = cb_entry.named_steps["prep"]
    cb_model = cb_entry.named_steps["clf"]
else:
    prep     = preprocessor
    cb_model = cb_entry

# 2) Preprocess your training set into the numeric array
X_train_pre = prep.transform(X_train)

# 3) Reconstruct the output feature names (cont + one-hot cats)
ohe            = prep.named_transformers_["cat"].named_steps["onehot"]
cat_ohe_names  = ohe.get_feature_names_out(cat)
feat_names     = cont + list(cat_ohe_names)

# 4) Compute mean(|SHAP|) per pre‐processed feature and grab top 3 indices
mean_abs_shap = np.abs(shap_values_cb).mean(axis=0)
top3_idx      = np.argsort(mean_abs_shap)[-3:]

# 5) Plot partial dependence for the top 3
fig, axes = plt.subplots(1, 3, figsize=(12, 4))
PartialDependenceDisplay.from_estimator(
    cb_model,
    X_train_pre,
    features=list(top3_idx),    # integer indices into your preprocessed matrix
    feature_names=feat_names,   # labels for each column
    grid_resolution=50,
    ax=axes
)
plt.suptitle("Partial Dependence – CatBoost (top 3 features)", y=1.02)
plt.tight_layout()
plt.show()


In [ ]:
# ╔═╡ Cell 11.2 – Permutation importance (CatBoost) ─════════════════════╗
from sklearn.inspection import permutation_importance
import matplotlib.pyplot as plt
import pandas as pd

# 1) Figure out where the preprocessor & model live:
cb_pipe_candidate = models["CatBoost"]
if hasattr(cb_pipe_candidate, "named_steps"):
    # it’s a Pipeline(prep → clf)
    prep     = cb_pipe_candidate.named_steps["prep"]
    cb_model = cb_pipe_candidate.named_steps["clf"]
else:
    # it’s a bare CatBoostClassifier
    prep     = preprocessor
    cb_model = cb_pipe_candidate  # the classifier itself

# 2) Preprocess your test‐set once
X_test_pre = prep.transform(X_test)

# 3) Reconstruct feature names in the same order:
#    – continuous first
#    – then the OneHotEncoder names inside your `preprocessor`’s "cat" transformer
num_feats = cont
cat_ohe = prep.named_transformers_["cat"].named_steps["onehot"]
cat_ohe_names = cat_ohe.get_feature_names_out(cat)
feat_names    = num_feats + list(cat_ohe_names)

# 4) Compute permutation_importance on the _raw_ CatBoost model:
r = permutation_importance(
    cb_model,
    X_test_pre, y_test,
    scoring="roc_auc",
    n_repeats=20,
    random_state=RANDOM_STATE,
    n_jobs=-1
)

# 5) Build a DataFrame and plot the top 20
imp_df = pd.DataFrame({
    "feature":          feat_names,
    "mean_decrease_auc": r.importances_mean
}).sort_values("mean_decrease_auc", ascending=False).head(20)

plt.figure(figsize=(6, 8))
plt.barh(imp_df["feature"][::-1], imp_df["mean_decrease_auc"][::-1])
plt.xlabel("Mean decrease in AUC")
plt.title("Permutation importance – CatBoost")
plt.tight_layout()
plt.show()


In [ ]:
# ╔═╡ Cell 11.3 – Decision curve (CatBoost) ─────────────────────────────╗
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline

# 1) Figure out where the preprocessor lives and where the CatBoost lives
cb_obj = models["CatBoost"]
if isinstance(cb_obj, Pipeline):
    prep     = cb_obj.named_steps["prep"]
    cb_model = cb_obj.named_steps["clf"]
else:
    prep     = preprocessor    # your ColumnTransformer from Cell 4
    cb_model = cb_obj          # raw CatBoostClassifier

# 2) Turn X_test into the exact numeric matrix your model expects
X_test_num = prep.transform(X_test)

# 3) Get predicted probabilities for the positive class
probs = cb_model.predict_proba(X_test_num)[:, 1]

# 4) Build a simple “net benefit” curve
thresholds = np.linspace(0, 1, 100)
n          = len(y_test)
event_rate = y_test.mean()

nb = []
for t in thresholds:
    tp = ((probs >= t) & (y_test == 1)).sum()
    fp = ((probs >= t) & (y_test == 0)).sum()
    nb.append(tp/n - fp/n * (t/(1-t)))

plt.figure(figsize=(6, 4))
plt.plot(thresholds, nb,       label="CatBoost")
plt.plot(thresholds, np.zeros_like(thresholds), "--", label="Treat none")
plt.plot(
    thresholds,
    event_rate - (1-event_rate)*(thresholds/(1-thresholds)),
    ":", label="Treat all"
)
plt.xlabel("Decision threshold")
plt.ylabel("Net benefit")
plt.title("Decision Curve Analysis – CatBoost")
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# ╔═╡ Cell 11.4 – Final evaluation on future hold-out for CatBoost (fixed) ─════╗
from sklearn.metrics         import (
    roc_auc_score,
    confusion_matrix,
    classification_report,
    RocCurveDisplay,
    roc_curve
)
from sklearn.calibration     import calibration_curve
from sklearn.metrics         import brier_score_loss
import matplotlib.pyplot     as plt
import numpy as np

# 1) Unpack your “models['CatBoost']”, which might be either:
#    a) a Pipeline(prep→clf) or b) a raw CatBoostClassifier
cb_obj = models["CatBoost"]

if hasattr(cb_obj, "named_steps"):
    # (a) you stored a Pipeline
    pre = cb_obj.named_steps["prep"]    # your ColumnTransformer
    clf = cb_obj.named_steps["clf"]     # CatBoostClassifier
    # apply the same preprocessing
    X_val_num = pre.transform(X_test)
else:
    # (b) you stored a bare CatBoostClassifier
    clf = cb_obj
    # apply your preprocessor manually
    X_val_num = preprocessor.transform(X_test)

y_val = y_test

# 2) Predict
y_val_proba = clf.predict_proba(X_val_num)[:, 1]
y_val_pred  = (y_val_proba >= 0.5).astype(int)

# 3) Discrimination & classification metrics
print("CatBoost AUC:           ", roc_auc_score(y_val, y_val_proba))
print("CatBoost Confusion @0.5:\n", confusion_matrix(y_val, y_val_pred))
print("\nCatBoost Classification report:\n",
      classification_report(y_val, y_val_pred))

# 4) ROC curve
RocCurveDisplay.from_predictions(y_val, y_val_proba)
plt.title("CatBoost – ROC on hold-out")
plt.tight_layout()
plt.show()

# 5) Calibration curve + Brier score
frac_pos, mean_pred = calibration_curve(y_val, y_val_proba, n_bins=10)
plt.figure(figsize=(6, 4))
plt.plot(mean_pred, frac_pos, "o-", label="CatBoost hold-out")
plt.plot([0, 1], [0, 1], "k--", label="Perfect calibration")
plt.xlabel("Mean predicted probability")
plt.ylabel("Fraction of positives")
plt.title("CatBoost – Calibration on hold-out")
plt.legend()
plt.tight_layout()
plt.show()

print("CatBoost Brier score:", brier_score_loss(y_val, y_val_proba))

# 6) Optimal threshold via Youden’s J
fpr_cb, tpr_cb, thresh_cb = roc_curve(y_val, y_val_proba)
j_scores_cb = tpr_cb - fpr_cb
best_thresh_cb = thresh_cb[np.argmax(j_scores_cb)]
print(f"CatBoost optimal threshold (max J): {best_thresh_cb:.3f}")


In [ ]:
# ╔═╡ Cell 11.5 – Calibrate your CatBoost on the build set ═════════════╗
from sklearn.calibration import CalibratedClassifierCV
from sklearn.pipeline    import Pipeline

# a) unpack your existing prep→clf
cb_pipe = models["CatBoost"]
if hasattr(cb_pipe, "named_steps"):
    prep = cb_pipe.named_steps["prep"]
    base = cb_pipe.named_steps["clf"]
else:
    prep = preprocessor
    base = cb_pipe

# b) build a small Pipeline with isotonic (or sigmoid) calibration
calibrated_cb = Pipeline([
    ("prep", prep),
    ("cal",  CalibratedClassifierCV(
                 estimator=base,     # <— use 'estimator', not 'base_estimator'
                 method="isotonic",  # or 'sigmoid'
                 cv=3
             ))
])

# c) fit only on your 80% build data
calibrated_cb.fit(X_train, y_train)

# d) swap into your models dict
models["CatBoost_Calibrated"] = calibrated_cb

print("✅ CatBoost_Calibrated added to models dict")


In [ ]:
# ╔═╡ Cell 11.6 – Bootstrap‐ensemble for CatBoost (fixed) ─════════════╗
from sklearn.utils     import resample
from sklearn.metrics   import roc_auc_score
from catboost          import CatBoostClassifier
import numpy as np

# unpack your pipeline
cb_entry = models["CatBoost"]
if hasattr(cb_entry, "named_steps"):
    prep     = cb_entry.named_steps["prep"]
    base_cb  = cb_entry.named_steps["clf"]
else:
    prep     = preprocessor
    base_cb  = cb_entry

n_boot = 20
boot_preds = np.zeros((len(X_test), n_boot))

for i in range(n_boot):
    # 1) bootstrap‐resample the raw DataFrame + labels
    Xb_df, yb = resample(X_train, y_train, random_state=RANDOM_STATE + i)
    # 2) prepare_for_catboost (fills NaNs, casts cats to str)
    Xb_cb = prepare_for_catboost(Xb_df, cat)
    Xtest_cb = prepare_for_catboost(X_test, cat)
    # 3) clone the CatBoost with identical hyperparams
    cb = CatBoostClassifier(**base_cb.get_params())
    # 4) fit on the DataFrame (CatBoost will handle numeric & string cats)
    cb.fit(
      Xb_cb, yb,
      cat_features=cat_features_idx,
      verbose=False
    )
    # 5) predict on your hold‐out DataFrame
    boot_preds[:, i] = cb.predict_proba(Xtest_cb)[:, 1]

# aggregate
y_mean = boot_preds.mean(axis=1)
print("Bootstrapped CatBoost AUC:", roc_auc_score(y_test, y_mean))


In [ ]:
# ╔═╡ Cell 11.7 – Compare Calibrated vs Bootstrap CatBoost on future hold‐out ─══╗
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics       import (
    roc_auc_score,
    confusion_matrix,
    classification_report,
    RocCurveDisplay,
    roc_curve,
    brier_score_loss
)
from sklearn.calibration   import calibration_curve
from sklearn.utils         import resample
from catboost              import CatBoostClassifier

# ------------------------------------------------------------------------------
# 1) Calibrated CatBoost evaluation
# ------------------------------------------------------------------------------
cal_cb = models["CatBoost_Calibrated"]
X_val, y_val = X_test, y_test

# predict
y_cal_proba = cal_cb.predict_proba(X_val)[:, 1]
y_cal_pred  = cal_cb.predict(X_val)

# metrics
print("=== Calibrated CatBoost on hold‐out ===")
print("AUC:           ", roc_auc_score(y_val, y_cal_proba))
print("Confusion @0.5:\n", confusion_matrix(y_val, y_cal_pred))
print(classification_report(y_val, y_cal_pred))

# ROC
RocCurveDisplay.from_predictions(y_val, y_cal_proba)
plt.title("Calibrated CatBoost – ROC")
plt.tight_layout()
plt.show()

# Calibration curve + Brier
frac_pos_c, mean_pred_c = calibration_curve(y_val, y_cal_proba, n_bins=10)
plt.figure(figsize=(6,4))
plt.plot(mean_pred_c, frac_pos_c, "o-", label="Calibrated")
plt.plot([0,1],[0,1],"k--", label="Ideal")
plt.xlabel("Mean predicted probability")
plt.ylabel("Fraction of positives")
plt.title("Calibrated CatBoost – Calibration")
plt.legend()
plt.tight_layout()
plt.show()

print("Brier score:", brier_score_loss(y_val, y_cal_proba))
print()

# ------------------------------------------------------------------------------
# 2) Bootstrap‐ensemble CatBoost evaluation
# ------------------------------------------------------------------------------
# unpack raw CatBoost
cb_entry = models["CatBoost"]
if hasattr(cb_entry, "named_steps"):
    prep    = cb_entry.named_steps["prep"]
    base_cb = cb_entry.named_steps["clf"]
else:
    prep    = preprocessor
    base_cb = cb_entry

n_boot = 20
boot_preds = np.zeros((len(X_val), n_boot))
for i in range(n_boot):
    Xb_df, yb = resample(X_train, y_train, random_state=RANDOM_STATE+i)
    Xb_cb = prepare_for_catboost(Xb_df, cat)
    Xval_cb = prepare_for_catboost(X_val, cat)
    cb = CatBoostClassifier(**base_cb.get_params())
    cb.fit(Xb_cb, yb, cat_features=cat_features_idx, verbose=False)
    boot_preds[:, i] = cb.predict_proba(Xval_cb)[:, 1]

y_boot_mean = boot_preds.mean(axis=1)

print("=== Bootstrap‐ensemble CatBoost on hold‐out ===")
print("AUC:           ", roc_auc_score(y_val, y_boot_mean))
# hard‐classify at 0.5
y_boot_pred = (y_boot_mean >= 0.5).astype(int)
print("Confusion @0.5:\n", confusion_matrix(y_val, y_boot_pred))
print(classification_report(y_val, y_boot_pred))

# ROC
RocCurveDisplay.from_predictions(y_val, y_boot_mean)
plt.title("Bootstrap CatBoost – ROC")
plt.tight_layout()
plt.show()

# Calibration curve + Brier
frac_pos_b, mean_pred_b = calibration_curve(y_val, y_boot_mean, n_bins=10)
plt.figure(figsize=(6,4))
plt.plot(mean_pred_b, frac_pos_b, "o-", label="Bootstrapped")
plt.plot([0,1],[0,1],"k--", label="Ideal")
plt.xlabel("Mean predicted probability")
plt.ylabel("Fraction of positives")
plt.title("Bootstrap CatBoost – Calibration")
plt.legend()
plt.tight_layout()
plt.show()

print("Brier score:", brier_score_loss(y_val, y_boot_mean))
